## Aggregate locations of dense [AIS broadcast reports](https://marinecadastre.gov/ais) at the port of Miami, FL.

In [3]:
gdb_path = os.path.expanduser(os.path.join("~", "data", "Miami.gdb"))

In [4]:
df = spark.read.format("gdb").options(path=gdb_path).load().show(truncate=False)

23/10/11 09:49:51 DEBUG GDBIndex$: GDBHeader:version=3, numPages=1, numRows=14, numBytesPerRow=5, bitMapOffset=5136, maxRows=1024
23/10/11 09:49:51 DEBUG GDBIndex$: GDBBitmap:size=0, numBits=1, numBitsSet=1, lastBit=0
23/10/11 09:49:51 DEBUG GDBTable$: Caching '/Users/mraad/data/Miami.gdb/a00000001.gdbtable'...
23/10/11 09:49:51 DEBUG GDBTable$: numFeatures=14
23/10/11 09:49:51 DEBUG GDBTable$: geometryType=0 (Unknown)
23/10/11 09:49:51 DEBUG GDBTable$: hasZ=false hasM=false
23/10/11 09:49:51 DEBUG GDBTable$: numFields=3
23/10/11 09:49:52 DEBUG GDBRDD: Max rows per partition=14
23/10/11 09:49:52 DEBUG GDBRDD: startAtRow=0 numRowsToRead=14


+---+-------------------------+----------+
|ID |Name                     |FileFormat|
+---+-------------------------+----------+
|1  |GDB_SystemCatalog        |0         |
|2  |GDB_DBTune               |0         |
|3  |GDB_SpatialRefs          |0         |
|4  |GDB_Items                |0         |
|5  |GDB_ItemRelationships    |0         |
|6  |GDB_ItemRelationshipTypes|0         |
|7  |GDB_ItemTypes            |0         |
|8  |GDB_ReplicaLog           |2         |
|9  |MiamiExtent              |0         |
|10 |Voyage                   |0         |
|11 |Broadcast                |0         |
|12 |Vessel                   |0         |
|13 |BaseStations             |0         |
|14 |AttributeUnits           |0         |
+---+-------------------------+----------+



### Create a Spark Dataframe and register it as a table for upcoming SQL operations

In [5]:
df = spark.read.format("gdb").options(path=gdb_path, name="Broadcast").load()

23/10/11 09:50:10 DEBUG GDBTable$: Caching '/Users/mraad/data/Miami.gdb/a0000000b.gdbtable'...
23/10/11 09:50:10 DEBUG GDBTable$: numFeatures=1365578
23/10/11 09:50:10 DEBUG GDBTable$: geometryType=1 (Point)
23/10/11 09:50:10 DEBUG GDBTable$: hasZ=false hasM=false
23/10/11 09:50:10 DEBUG GDBTable$: numFields=12
23/10/11 09:50:10 DEBUG GDBTable$: {"ymax":25.950708000000006,"srsWKT":"GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]","xmin":-80.282093,"ymin":25.591130000000007,"xmax":-79.75126,"hasM":false,"alias":"Shape","hasZ":false,"geomType":1}


In [6]:
df.printSchema()

root
 |-- OBJECTID: integer (nullable = false)
 |-- Shape: struct (nullable = true)
 |    |-- x: double (nullable = true)
 |    |-- y: double (nullable = true)
 |-- SOG: integer (nullable = true)
 |-- COG: integer (nullable = true)
 |-- Heading: integer (nullable = true)
 |-- ROT: integer (nullable = true)
 |-- BaseDateTime: timestamp (nullable = true)
 |-- Status: integer (nullable = true)
 |-- VoyageID: integer (nullable = true)
 |-- MMSI: integer (nullable = true)
 |-- ReceiverType: string (nullable = true)
 |-- ReceiverID: string (nullable = true)



In [7]:
df.schema["Shape"].metadata

{'ymax': 25.950708000000006,
 'srsWKT': 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
 'xmin': -80.282093,
 'ymin': 25.591130000000007,
 'xmax': -79.75126,
 'hasM': False,
 'alias': 'Shape',
 'hasZ': False,
 'geomType': 1}

In [8]:
df.select("OBJECTID").write.parquet("/tmp/tmp.prq", mode="overwrite")

23/10/11 09:51:11 DEBUG GDBIndex$: GDBHeader:version=3, numPages=11568, numRows=11845541, numBytesPerRow=5, bitMapOffset=59228176, maxRows=11845632
23/10/11 09:51:11 DEBUG GDBIndex$: GDBBitmap:size=0, numBits=11568, numBitsSet=11568, lastBit=0
23/10/11 09:51:11 DEBUG GDBRDD: Max rows per partition=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=0 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=592278 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=1184556 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=1776834 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=2369112 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=2961390 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=3553668 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=4145946 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=4738224 numRowsToRead=592278
23/10/11 09:51:11 DEBUG GDBRDD: startAtRow=5330502 nu

1365578

In [10]:
spark.read.parquet("/tmp/tmp.prq").count()

1365578